## Análisis de bases de datos  
<b>Taller 4: Análisis multidimensional de criminalidad </b>  
Laura Nicole Bermudez Santa - labermudezs@unal.edu.co

<p>Considere los datos del archivo Excel adjunto; en él se muestran los datos de criminalidad, fondos policiales y educación de la población en las ciudades pequeñas de los Estados Unidos.</p>
<p>Las variables <b>(X1, X2, X3, X4, X5, X6, X7)</b> representan la siguiente información:</p>  

- <b>X1:</b> reporte total de criminalidad por millón de residentes.   

- <b>X2:</b> tasa de crímenes violentos por 100.000 residentes.  

- <b>X3:</b> fondos anuales policiales en dólares por habitante.  

- <b>X4:</b> porcentaje de personas de 25 años o más con bachillerato.    

- <b>X5:</b> porcentaje de la población de 16 a 19 años sin bachillerato.    

- <b>X6:</b> porcentaje de la población entre 18 a 24 años que realiza estudios universitarios.  

- <b>X7:</b> porcentaje de la población de 25 o más años con por lo menos 4 años de estudios universitarios.    

In [1]:
# Importamos algunas librerias necesarias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sb

# Leemos el archivo
criminalidad = pd.read_csv('Datos Criminalidad.csv')
criminalidad.head(5)

,X1,X2,X3,X4,X5,X6,X7
0,478,184,40,74,11,31,20
1,494,213,32,72,11,43,18
2,643,347,57,70,18,16,16
3,341,565,31,71,11,25,19
4,773,327,67,72,9,29,24


In [2]:
criminalidad.info() # Información basica de las columnas que conforman el dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   X1      50 non-null     int64
 1   X2      50 non-null     int64
 2   X3      50 non-null     int64
 3   X4      50 non-null     int64
 4   X5      50 non-null     int64
 5   X6      50 non-null     int64
 6   X7      50 non-null     int64
dtypes: int64(7)
memory usage: 2.9 KB


<b>A.</b> Presente un análisis estadístico básico por variable indicando sus opiniones sobre cada uno de los valores calculados. En este literal debe efectuar el cálculo de todas las medidas vistas en clase de centralización y dispersión, debe incluir un análisis intercuartílico.

In [3]:
criminalidad.describe() # Estadisticas basicas de las columnas que conforman el dataset

,X1,X2,X3,X4,X5,X6,X7
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,717.960000,616.180000,37.760000,58.800000,15.400000,29.900000,13.820000
std,293.938766,573.739175,13.820364,9.965246,6.023762,14.801062,5.157479
min,341.000000,29.000000,16.000000,42.000000,4.000000,7.000000,8.000000
25%,497.000000,230.750000,30.000000,49.000000,11.000000,21.250000,11.000000
50%,654.500000,454.000000,34.500000,59.000000,14.000000,25.000000,12.000000
75%,820.500000,822.500000,42.250000,67.000000,19.000000,34.250000,15.750000
max,1740.000000,3545.000000,86.000000,81.000000,34.000000,81.000000,36.000000


La tabla anterior presenta un análisis estadístico básico por cada variable contemplando los 50 registros del dataset. Se expone en su orden la cantidad de datos no nulos evaluados, la media aritmética, la desviación estandar, el valor mínimo, primer y tercer cuartil, mediana y por ultimo el valor máximo.

<b>B.</b> Presente una matriz de "calor" para las correlaciones (de Person) entre pares de variables. Indique cuales son los cuatro pares de variables con mayor correlación.

<b>C.</b> Plotee los cuatro pares de variables con mayor correlación, mostrando la recta de regresión lineal que mejor se ajusta a los datos.

<b>D.</b> Haga un análisis multilineal tomando como variable dependiente los fondos anuales policiales por habitante, con las demás variables como variables predictorias. Indique la expresión que obtuvo.

<b>E.</b> Indique que variables predictorias tiene mayor impacto sobre la variable independinte. ¿Tiene sentido lo obtenido? Explique.

<b>F.</b> Haga un análisis de regresión lineal simple entre la variable fondos anuales reales y los predichos por el modelo. Con base en este análisis evalúe el modelo.

<b>G.</b> Plantee un modelo de regresión multilineal que permita explicar los índices de criminalidad con el grado de escolaridad de la población. Explique brevemente su modelo y los resultados que obtiene de él.

<b>H.</b> ¿Es posible detectar datos atípicos en la base de datos con las herramientas vistas en clase?